In [1]:
!pip install -q pinecone-client sentence-transformers requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.6 MB/s eta 0:00:00


In [6]:
!pip uninstall -y pinecone-client
!pip install -U pinecone


Found existing installation: pinecone-client 6.0.0
Uninstalling pinecone-client-6.0.0:
  Successfully uninstalled pinecone-client-6.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 24.2 MB/s eta 0:00:00


In [7]:
import os
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer

PINECONE_API_KEY = "pcsk_5qM3R_RTNpiTNe6KMSLvYicdZvzc48WR8zXAGYjVH6iozaCeUYkydMhmw42toBHwVhQCJ"
PINECONE_ENV = "us-east-1"

pc = Pinecone(api_key=PINECONE_API_KEY)
model = SentenceTransformer("all-MiniLM-L6-v2")  # 384-dim


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from pinecone import ServerlessSpec

index_name = "imdb-movie-index-test"

# Delete existing (optional)
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)


{
    "name": "imdb-movie-index-test",
    "metric": "cosine",
    "host": "imdb-movie-index-test-ta01y6m.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [9]:
import pandas as pd

columns = [
    "tconst", "titleType", "primaryTitle", "originalTitle", "isAdult",
    "startYear", "endYear", "runtimeMinutes", "genres"
]

url = "https://datasets.imdbws.com/title.basics.tsv.gz"
df = pd.read_csv(url, sep="\t", na_values="\\N", compression="gzip", names=columns, header=0)
df = df[df["titleType"] == "movie"].dropna(subset=["startYear", "genres"])
df = df[df["startYear"].astype(int).between(2000, 2020)]
df = df[["tconst", "primaryTitle", "startYear", "genres"]].head(100)
df["genre"] = df["genres"].apply(lambda x: x.split(",")[0].lower())
df = df.rename(columns={"tconst": "id", "primaryTitle": "title", "startYear": "year"})
df = df[["id", "title", "year", "genre"]]


/tmp/ipython-input-9-2975613443.py:9: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, sep="\t", na_values="\\N", compression="gzip", names=columns, header=0)


In [11]:
from tqdm.notebook import tqdm

df["plot"] = df["title"] + " (simulated description)"
df["embedding"] = df["plot"].apply(lambda x: model.encode(x, normalize_embeddings=True).tolist())

index = pc.Index(index_name)

# Insert grouped by genre
for genre, group in df.groupby("genre"):
    vectors = [
        (row["id"], row["embedding"], {
            "title": row["title"],
            "year": str(row["year"]),
            "genre": row["genre"]
        })
        for _, row in group.iterrows()
    ]
    index.upsert(vectors=vectors, namespace=genre)


In [12]:
def semantic_search(query, namespace, top_k=3):
    query_vector = model.encode(query, normalize_embeddings=True).tolist()
    results = index.query(vector=query_vector, top_k=top_k, include_metadata=True, namespace=namespace)

    if results and results.matches:
        for i, match in enumerate(results.matches, 1):
            metadata = match.metadata
            print(f"🔹 {i}. {metadata.get('title')} ({metadata.get('year')}) | Score: {match.score:.2f}")
    else:
        print("❌ No matches found.")


In [13]:
semantic_search("show me movies similar to space odyssey but made after 2010", namespace="adventure")


🔹 1. Dune 7 (2002.0) | Score: 0.33
🔹 2. Shrek (2001.0) | Score: 0.29
🔹 3. The Lord of the Rings: The Fellowship of the Ring (2001.0) | Score: 0.28
